TODO: Split the annotating of mutations with conditions and the associating of genomic features with conditions into 2 NBs.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.alemutdf import get_all_sample_mut_df, get_gene_mut_count_mat, get_multi_exp_max_freq_mut_df, get_mut_type_avg_frac_across_class_df
from util.metadata import get_condition_val_dict, get_condition_field_val_set
from util.genome import get_K12_pos_from_BOP27
from util.params import ASSOC_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [2]:
# starting efforts for refactoring of assoc NBs.
FEAT_ANNOT = "genomic features"

In [3]:
all_muts_df = pd.read_pickle("./data/2_4_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(1278, 31)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{}
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{}
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{}
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']}
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'NagC'}, {'name': 'CRP'}, {'name': '...","{'PhoP': ['nagBAC-umpH'], 'NagC': ['nagBAC-ump..."


## enrichment analysis between conditions and mutated genomic features

### Get metadata

In [4]:
from util.metadata import get_all_exp_cond_d

all_exp_cond_d = get_all_exp_cond_d("./data/metadata/")
all_exp_cond_d

{'SSW_XYL': {'taxonomy-id': '511145',
  'strain-description': 'WT',
  'temperature': '37 celsius',
  'base-media': 'M9',
  'carbon-source': 'xylose(4)',
  'nitrogen-source': 'NH4Cl(1)',
  'phosphorous-source': 'KH2PO4(3) Na2HPO4(6.8)',
  'sulfur-source': 'MgSO4(0.24)',
  'supplement': 'NaCl(0.5g/L) trace elements',
  'calcium-source': 'CaCl2(0.1)'},
 'Central_carbon_knockout_gnd': {'taxonomy-id': '511145',
  'strain-description': 'Δgnd',
  'base-media': 'M9',
  'temperature': '37 celsius',
  'carbon-source': 'glucose(4)',
  'nitrogen-source': 'NH4Cl(1)',
  'phosphorous-source': 'KH2PO4(3) Na2HPO4(6.8)',
  'sulfur-source': 'MgSO4(0.24)',
  'supplement': 'NaCl(0.5g/L) trace elements',
  'calcium-source': 'CaCl2(0.1)'},
 'Central_carbon_knockout_pgi': {'taxonomy-id': '511145',
  'strain-description': 'Δpgi',
  'base-media': 'M9',
  'temperature': '37 celsius',
  'carbon-source': 'glucose(4)',
  'nitrogen-source': 'NH4Cl(1)',
  'phosphorous-source': 'KH2PO4(3) Na2HPO4(6.8)',
  'sulfur-sour

In [5]:
# So that rows will always contain the same amount of columns
all_muts_df = all_muts_df.replace('', "None")
display(all_muts_df.shape, all_muts_df.head())

(1278, 31)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{}
1,6,None,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{}
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{}
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']}
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'NagC'}, {'name': 'CRP'}, {'name': '...","{'PhoP': ['nagBAC-umpH'], 'NagC': ['nagBAC-ump..."


### mutation to condition association

In [6]:
# # needs to be a list of UNIQUE columns
# test_s = set()
# for d in all_exp_cond_d.values():
#     test_s = test_s | set(d.keys())
# display(test_s)

In [7]:
from util.metadata_categories_for_associations import METADATA_CATEGORIES_FOR_ASSOCIATIONS

for condition in METADATA_CATEGORIES_FOR_ASSOCIATIONS:
    all_muts_df[condition] = all_muts_df["exp"].apply(lambda exp: all_exp_cond_d[exp][condition])
    
all_muts_df = all_muts_df.reset_index(drop=True)

In [8]:
# The execution of get_ALE_max_freq_mut_df(get_endpoints_only=True) earlier in the pipeline should have already removed all duplicate mutations
# Commenting the old logic (above) out. To use only when using more than just endpoints. Annotated in NB1 for the need to uncomment this region is not using endpoints.
# This code is inappropriate anyways because you can have two different and unique mutations to the same gene though with different position, or same position though with different sequence changes.
# Important to note that associations are made on the resolution of genomic feature and not position, though two mutations with different detail though the same feature will be represented, as currently expected.
# The logic uncommented above is therefore incomplete for removing duplicates for sample sets that include midpoints and endpoints.

# The below is the necessary logic without this filtering.
exp_target_cond_df = all_muts_df.copy()
display(exp_target_cond_df.shape, exp_target_cond_df.head())

(1278, 41)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1,6,None,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']},42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'NagC'}, {'name': 'CRP'}, {'name': '...","{'PhoP': ['nagBAC-umpH'], 'NagC': ['nagBAC-ump...",42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


In [9]:
unique_condition_set = set()
for cond_cat in METADATA_CATEGORIES_FOR_ASSOCIATIONS:
    unique_condition_set |= set(exp_target_cond_df[cond_cat].unique())
unique_condition_set

{'37 celsius',
 '42 celsius',
 '511145',
 'CaCl2(0.1)',
 'GMOS',
 'KH2PO4(3) Na2HPO4(6.8)',
 'KH2PO4(6) Na2HPO4(13.6)',
 'M9',
 'MgSO4(0.24)',
 'MgSO4(24)',
 'NH4Cl(1)',
 'NH4Cl(2)',
 'NaCl(0.5g/L) trace elements',
 'NaCl(1g/L) trace elements',
 'U-13C glucose(2)',
 'WT',
 'acetate(4)',
 'glucose(2)',
 'glucose(4)',
 'glucose(4) or acetate(4)',
 'glucose(4) or glycerol(2.5)',
 'glucose(4) or xylose(4)',
 'glycerol(2.5)',
 "glycine(2mM)  L-Serine(varying concentration)  trace elements(X1)  Wolfe's vitamin solution.",
 'xylose(4)',
 'Δgnd',
 'Δpgi',
 'Δpts',
 'ΔsdaA ΔsdaB ΔtdcG ΔglyA',
 'Δsdh',
 'Δtpi'}

In [10]:
feat_cond_df = pd.DataFrame()
for _, r in exp_target_cond_df.iterrows():
    for feat in r[FEAT_ANNOT]:
        for cond_col in METADATA_CATEGORIES_FOR_ASSOCIATIONS:
            if r[cond_col] in unique_condition_set:   # I'm not quite sure why I had to check this in the legacy code.
                feat_cond_df = feat_cond_df.append({"feature": feat["name"], "condition":r[cond_col]}, ignore_index=True)
display(len(feat_cond_df), feat_cond_df.head())

15310

,condition,feature
0,42 celsius,clsA
1,glucose(4),clsA
2,NaCl(0.5g/L) trace elements,clsA
3,WT,clsA
4,511145,clsA


TODO: Now that this logic is being used for multiple feature types (therefore multiple times), should extract it into a util script file

In [11]:
from statsmodels.stats import multitest
import scipy


def get_contingency_table(count_mat, row_name, col_name):
    row_col_count = count_mat.loc[row_name, col_name]
    row_not_col_sum_count = count_mat.loc[row_name].sum() - row_col_count
    not_row_col_sum_count = count_mat.T.loc[col_name].sum() - row_col_count
    all_mat_sum_count = count_mat.sum(axis=1).sum()
    not_row_not_col_sum_count = all_mat_sum_count - \
        row_col_count - row_not_col_sum_count - not_row_col_sum_count
    contingency_table = [
        [row_col_count, row_not_col_sum_count],
        [not_row_col_sum_count, not_row_not_col_sum_count]
    ]
    return contingency_table


test_df = pd.DataFrame([[1, 2, 3],
                        [4, 5, 6]],
                       index=["y1", "y2"],
                       columns=["x1", "x2", "x3"])
contingency_table = get_contingency_table(test_df, "y2", "x3")
expected_contingency_table = [[6, 9], [3, 3]]
assert(contingency_table == expected_contingency_table)


def get_multiple_hypothesis_correction(pval_df):
    # Dataframe seemed to be designed to iterate by column more easily than row (see df.iteritems())
    # Have to combine all p-value columns because multitest.multipletests only takes 1D array inputs.
    pval_l = []
    for col in list(pval_df.columns.values):
        pval_l += list(pval_df[col])
    corrected_pval_result = multitest.multipletests(
        pvals=pval_l,
        alpha=ASSOC_ALPHA,
        method=MULTI_HYP_CORR_METHOD)

    # The following splits the multitest.multipletests 1D array output into the same shape, though transposed.
    corrected_pval_l = corrected_pval_result[1]
    df_col_len = pval_df.shape[0]
    corrected_pval_mat = [corrected_pval_l[i:i+df_col_len]
                          for i in range(0, len(corrected_pval_l), df_col_len)]

    # building df with new p-values
    corrected_pval_df = pval_df.copy()
    for outer_l_idx in range(0, len(corrected_pval_df.columns)):
        for inner_l_idx in range(0, len(corrected_pval_df)):
            corrected_pval = corrected_pval_mat[outer_l_idx][inner_l_idx]
            corrected_pval_df.iloc[inner_l_idx, outer_l_idx] = corrected_pval
    return corrected_pval_df


# def get_enrich_genetic_target_df(cond_col, mut_df):
def get_enrich_genetic_target_df(feat_cond_df):
    # Creates tables of counts between each mutation and unique condition.
    cross_counts_df = pd.crosstab(
        feat_cond_df["feature"], feat_cond_df["condition"]
    )

    # To reuse the same DF indeces without having to remake it.
    enrich_odds_ratio_df = cross_counts_df.copy()
    enrich_pvals_df = cross_counts_df.copy()

    for cond in cross_counts_df.columns.values:
        for mut_target in cross_counts_df.index:
            contingency_table = get_contingency_table(cross_counts_df, mut_target, cond)
            odds_ratio, p_val = scipy.stats.fisher_exact(contingency_table, alternative="greater")
            enrich_odds_ratio_df.loc[mut_target, cond] = odds_ratio
            enrich_pvals_df.loc[mut_target, cond] = p_val

    enriched_pvals_df = get_multiple_hypothesis_correction(enrich_pvals_df)

    return enrich_odds_ratio_df, enrich_pvals_df

In [12]:
enrich_odds_ratio_df, enrich_pvals_df = get_enrich_genetic_target_df(feat_cond_df)

In [13]:
signif_genomic_feat_cond_json = []
for (mut_serial, row) in enrich_pvals_df.iterrows():
    for cond in row.index:
        p_val = enrich_pvals_df.at[mut_serial, cond]
        odds_ratio = enrich_odds_ratio_df.at[mut_serial, cond]
        if odds_ratio > 1 and p_val < ASSOC_ALPHA:
            signif_genomic_feat_cond_json.append({"mutated features": mut_serial, "condition": cond, "odd ratio": odds_ratio, "p value":p_val})

In [14]:
import pickle
f = open("./data/signif_genomic_feat_cond_json.pkl", 'wb')
pickle.dump(signif_genomic_feat_cond_json, f)

In [15]:
signf_assoc_feat_cond_d = {d["mutated features"]:set() for d in signif_genomic_feat_cond_json}
for d in signif_genomic_feat_cond_json:
    target = d["mutated features"]
    condition = d["condition"]
    if target in signf_assoc_feat_cond_d.keys():
        signf_assoc_feat_cond_d[target].add(condition)
    else:
        signf_assoc_feat_cond_d[target] = {condition}
signf_assoc_feat_cond_d

{'araA': {'Δpts'},
 'cpxR': {'KH2PO4(6) Na2HPO4(13.6)',
  'MgSO4(24)',
  'NH4Cl(2)',
  'NaCl(1g/L) trace elements',
  'glucose(2)'},
 'crr': {'KH2PO4(6) Na2HPO4(13.6)',
  'MgSO4(24)',
  'NH4Cl(2)',
  'NaCl(1g/L) trace elements',
  'glucose(2)',
  'glycerol(2.5)'},
 'csgDEFG TFBS': {'42 celsius'},
 'cyaA': {'glycerol(2.5)'},
 'cyoAp': {'GMOS'},
 'cysDp': {'Δsdh'},
 'fliK': {'acetate(4)'},
 'gcvRp': {'glucose(2)',
  "glycine(2mM)  L-Serine(varying concentration)  trace elements(X1)  Wolfe's vitamin solution.",
  'ΔsdaA ΔsdaB ΔtdcG ΔglyA'},
 'glkp': {'Δpts'},
 'glkp1': {'Δpts'},
 'glnD': {'Δsdh'},
 'glpK': {'glycerol(2.5)'},
 'glpR': {'glucose(4) or glycerol(2.5)'},
 'hisLGDCBHAFI attenuator terminator': {'glycerol(2.5)'},
 'icd': {'KH2PO4(6) Na2HPO4(13.6)',
  'MgSO4(24)',
  'NH4Cl(2)',
  'NaCl(1g/L) trace elements',
  'acetate(4)',
  'glucose(2)',
  'glucose(4) or acetate(4)',
  'Δpgi'},
 'ilvG_1': {'GMOS'},
 'ilvLXG_1G_2MEDA terminator': {'Δgnd'},
 'leuLABCD attenuator terminator': {'gl

In [16]:
def _get_feat_signf_assoc_cond_set(feat_d, mut_conds):
    feat_signf_assoc_cond_set = set()
    if feat_d["name"] in signf_assoc_feat_cond_d.keys():
        
        # Only includes conditions from exp mut comes from, therefore may be subset of signf assoc cond.
        for c in signf_assoc_feat_cond_d[feat_d["name"]]:
            if c in mut_conds:
                feat_signf_assoc_cond_set.add(c)
    
    return feat_signf_assoc_cond_set


def get_feat_signf_assoc_cond_set_json(mut_df_row, feat_annot_name):
    feat_json = []
    for d in mut_df_row[feat_annot_name]:
        feat_d = d.copy()
        mut_conds = list(mut_df_row[METADATA_CATEGORIES_FOR_ASSOCIATIONS])
        feat_d["significantly associated conditions"] = _get_feat_signf_assoc_cond_set(d, mut_conds)
        feat_json.append(feat_d)
    return feat_json


exp_target_cond_df[FEAT_ANNOT] = exp_target_cond_df.apply(lambda r: get_feat_signf_assoc_cond_set_json(r, FEAT_ANNOT), axis=1)

In [17]:
exp_target_cond_df.to_pickle("./data/4_df.pkl")
exp_target_cond_df.shape

(1278, 41)